## Import Required Libraries
Import the necessary libraries, including os, PIL, and matplotlib.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image
import matplotlib.pyplot as plt
import math
import json
import pandas as pd
import numpy as np
import torch
import sys
import csv
from tqdm import tqdm
import numpy as np

dataset_path = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\rawframes'

## Experimenting with the data

### Helper Functions

In [ ]:
def load_json(json_file_path):
    # open and load the JSON file
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    return data

def load_images(folder_path, num_images=None):
    image_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                image_files.append(os.path.join(root, file))
                if (num_images) and (len(image_files) == num_images):
                    break
    return image_files

def display_images(image_files):
    for i, image_file in enumerate(image_files):
        img = Image.open(image_file)
        plt.figure(figsize=(3, 3))
        plt.imshow(img )
        plt.title(os.path.basename(image_file))
        plt.axis('off')
        plt.show()

def load_and_display_images(folder_path,  num_images=None):
    """
    Load and display images from the specified folder.

    :param folder_path: Path to the folder containing images.
    :param num_images: Number of images to display.
    """
    image_files = load_images(folder_path, num_images)
    display_images(image_files)


def load_and_display_images_with_bb(training_json_dict, video_id, num_images=None, save_folder=None, display_images=True):
    training_keys = list(training_json_dict.keys())
    dict_image_data = training_json_dict[training_keys[2]]
    dict_annotation_data = training_json_dict[training_keys[3]]

    images_df = pd.DataFrame(dict_image_data)
    images_df = images_df[images_df['video_id'] == video_id]

    annotation_df = pd.DataFrame(dict_annotation_data)

    images_list = list(images_df.file_name)
    if not num_images:
        num_images = len(images_list)

    image_arrays = []
    for i, image_name in tqdm(enumerate(images_list[:num_images])):
        image_id = int(images_df[images_df['file_name'] == image_name].id)
        bbox = list(annotation_df[annotation_df['image_id'] == image_id]['bbox'])[0]
        image_array = np.array(Image.open(os.path.join(dataset_path, image_name)))
        image_arrays.append(image_array)
        plt.figure(figsize=(10, 10))
        plt.imshow(image_array)
        plt.gca().add_patch(plt.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], edgecolor='green', facecolor='none', linewidth=2))
        plt.axis('off')
        
        if save_folder:
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)
            plt.savefig(os.path.join(save_folder, os.path.basename(image_name)))
        
        if display_images:
            plt.show()

def buv_json_to_csv(buv_json_file, output_csv_file, two_points_format=True, only_train_frames=False):	
    json_dict = load_json(buv_json_file)
    dict_keys = list(json_dict.keys())
    dict_image_data = json_dict[dict_keys[2]]
    dict_annotation_data = json_dict[dict_keys[3]]

    images_df = pd.DataFrame(dict_image_data)
    if only_train_frames:
        images_df = images_df[images_df['is_vid_train_frame'] == True]

    annotation_df = pd.DataFrame(dict_annotation_data)


    images_list = list(images_df.file_name)
    print('image_list_size: ', len(images_list))

    images_to_ignore_list = ['benign/x66ef02e7f1b9a0ef', 'benign/x63c9ba1377f35bf6', 'benign/x5a1c46ec6377e946']

    with open(output_csv_file, 'w', newline='') as csvfile:
        fieldnames = ['img_file', 'x1', 'y1', 'x2', 'y2', 'class_name']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
    
        for image_name in tqdm(images_list):

            ignore = False
            for image_to_ignore in images_to_ignore_list:
                if image_to_ignore in image_name:
                    ignore = True
            if ignore:
                print(f'Image {image_name} is ignored')
                continue

            image_id = int(images_df[images_df['file_name'] == image_name].id)
            bbox = list(annotation_df[annotation_df['image_id'] == image_id]['bbox'])[0]

            if two_points_format:
                writer.writerow({
                    'img_file': os.path.join(dataset_path, image_name),
                    'x1': bbox[0] if bbox[2]>=0 else bbox[0] + bbox[2],
                    'y1': bbox[1] if bbox[3]>=0 else bbox[1] + bbox[3],
                    'x2': bbox[0] + bbox[2] if bbox[2]>=0 else bbox[0],
                    'y2': bbox[1] + bbox[3] if bbox[3]>=0 else bbox[1],
                    'class_name': 'Lesion'
                })
            else:
                writer.writerow({
                            'img_file': os.path.join(dataset_path, image_name),
                            'x1': bbox[0],
                            'y1': bbox[1],
                            'x2': bbox[2],
                            'y2': bbox[3],
                            'class_name': 'Lesion'
                        })
   


### Displaying the data

In [ ]:
training_json_file = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\imagenet_vid_train_15frames.json'
training_csv_file = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\train_annotations_2.csv'

#buv_json_to_csv(training_json_file, training_csv_file, only_train_frames=True)

validation_json_file = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\imagenet_vid_val.json'
validation_csv_file = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\val_annotations.csv'

#buv_json_to_csv(validation_json_file, validation_csv_file)

In [ ]:
training_json_dict = load_json(r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\imagenet_vid_train_15frames.json')
video_id = 63
save_path = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\example_images\63'
load_and_display_images_with_bb(training_json_dict, video_id, save_folder=save_path, display_images=False)

folder_path = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\rawframes\benign\2cda21c3aab26332'
#load_and_display_images(folder_path)

### Experimenting with the .json data files

In [ ]:
import os
folder_path = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\rawframes\benign'
video_name = os.listdir(folder_path)
file_name = os.listdir(os.path.join(folder_path,video_name[0]))

training_json_dict = load_json(r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\imagenet_vid_train_15frames.json')
training_keys = list(training_json_dict.keys())
print(training_keys)

video_index = 0

video_name = training_json_dict[training_keys[1]][video_index]['name']
video_folder_path = os.listdir(os.path.join(dataset_path,video_name))

# print the loaded data
print(list(training_json_dict.keys()))
print(training_json_dict[list(training_json_dict.keys())[0]][0])
print(training_json_dict[list(training_json_dict.keys())[1]][0])
print(training_json_dict[list(training_json_dict.keys())[2]][29])
print(training_json_dict[list(training_json_dict.keys())[3]][0])

## Retinanet Evaluation on the data

In [ ]:
models_directory = r"C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\pytorch-retinanet\trained_networks"
sys.path.append('./pytorch-retinanet')

import cv2
import time
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms

from retinanet.dataloader import CocoDataset, CSVDataset, collater, Resizer, AspectRatioBasedSampler, UnNormalizer, Normalizer
from retinanet import csv_eval, coco_eval

csv_val = r"C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\val_annotations.csv"
csv_classes = r"C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\class_name.csv"
dataset_val = CocoDataset('../Miccai 2022 BUV Dataset', set_name='imagenet_vid_val', transform=transforms.Compose([Normalizer(), Resizer()]))
sampler_val = AspectRatioBasedSampler(dataset_val, batch_size=1, drop_last=False)
dataloader_val = DataLoader(dataset_val, num_workers=1, collate_fn=collater, batch_sampler=sampler_val)

In [ ]:
experiment_1_directory = os.path.join(models_directory, 'experiment_1')
experiment_2_directory = os.path.join(models_directory, 'experiment_2')
experiment_3_directory = os.path.join(models_directory, 'experiment_3')
experiment_4_directory = os.path.join(models_directory, 'experiment_4')
experiment_5_directory = os.path.join(models_directory, 'experiment_5')



def validate_and_save(experiment_directory, coco_metrics=True, plot_froc=False, validation_json_file=None):
    for model_name in os.listdir(experiment_directory):

        if not model_name.endswith('.pt'):
            continue

        print(model_name)
        if coco_metrics:
            output_json_file = os.path.join(experiment_directory, f'{model_name}_coco_metrics.json')
        else:
            output_json_file = os.path.join(experiment_directory, f'{model_name}_metrics.json')

        model_path = os.path.join(experiment_directory, model_name)
        retinanet = torch.load(model_path)
        if coco_metrics:
            if not validation_json_file:
                raise ValueError('validation_json_file must be provided when using COCO metrics')
            #metrics_list = csv_eval.evaluate_coco_metrics(dataset_val, retinanet, buv_json_file=validation_json_file, model_path=model_path),
            metrics_list = coco_eval.evaluate_coco(dataset_val, retinanet, model_path=model_path),
        else:
            metrics_list = csv_eval.evaluate(dataset_val,retinanet,iou_threshold=0.5,score_threshold=0.05,max_detections=1,save_path=None)
        
        if not os.path.exists(output_json_file):
            with open(output_json_file, 'w') as f:
                json.dump(metrics_list, f, indent=4)

        if plot_froc:
            thresholds = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
            csv_eval.evaluate_froc(dataset_val, retinanet, buv_json_file=validation_json_file, model_path=model_path, score_thresholds=thresholds)

In [ ]:
validation_json_file = r'C:\Users\zaid-\OneDrive\Desktop\PMSD\Implementation\Miccai 2022 BUV Dataset\imagenet_vid_val.json'

In [ ]:
validate_and_save(models_directory, validation_json_file=validation_json_file)
#validate_and_save(experiment_2_directory, validation_json_file=validation_json_file)

In [5]:
import torch
model_path = 'C:/Users/zaid-/OneDrive/Desktop/PMSD/Implementation/pytorch-retinanet/trained_networks/experiment_4/coco_retinanet_13'
model = model_path + '.pt'
model = torch.load(model)
torch.save(model.state_dict(), f'{model_path}_model_weights.pt')